In [1]:
# simple data manipulation
import numpy  as np
import pandas as pd

# deep learning
import torch
import torch.nn as nn
import torch.nn.functional as F

# results logging
import wandb

# progress bar
from   tqdm.notebook import tqdm, trange

# remove warnings (remove deprecated warnings)
import warnings
warnings.simplefilter('ignore')

# visualization of resultsa
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from   matplotlib.ticker import MaxNLocator
import seaborn           as sns

# Graph Algorithms.
import networkx as nx

# Google Colab (many lines are removed)
import os
import zipfile
# from google.colab import drive
# from distutils.dir_util import copy_tree

# wheter we are using colab or not
COLAB: bool = False
if not COLAB and not os.path.exists('./data/simulations'): 
    os.chdir('..')

# Simulation Settings
from g6smart.sim_config import SimConfig
from g6smart.evaluation import rate as rate_metrics

config = SimConfig(0)
config

Simulation Parameters: 

|                      name |                     value |
---------------------------------------------------------
|        num_of_subnetworks |                   20.0000 |
|              n_subchannel |                    4.0000 |
|             deploy_length |                   20.0000 |
|             subnet_radius |                    1.0000 |
|                      minD |                    0.8000 |
|               minDistance |                    2.0000 |
|                 bandwidth |             40000000.0000 |
|              ch_bandwidth |             10000000.0000 |
|                        fc |           6000000000.0000 |
|                    lambdA |                    0.0500 |
|                  clutType |                     dense |
|                  clutSize |                    2.0000 |
|                  clutDens |                    0.6000 |
|                   shadStd |                    7.2000 |
|                 max_power |                  

## Simulations and Information

Thanks to the given scripts, we can load a group of generated simulations. They don't have any solutions (neither approximations).

In [2]:
# Moung Google Drive Code
if COLAB:
    # drive.mount('/content/drive')

    # Move Simulations to avoid cluttering the drive folder
    # if not os.path.exists('/content/simulations'):
    #   os.mkdir('/content/simulations')

    # if list(os.listdir('/content/simulations')) == []:
    #   copy_tree('/content/drive/MyDrive/TFM/simulations', '/content/simulations')

    # unzip all simulations
    # print("Name of the already simulated data: \n", )
    for zip_file in os.listdir('/content/simulations'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("/content/simulations/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('/content/simulations/')

    SIMULATIONS_PATH: str = "/content/simulations"
else:
    if not os.path.exists('./data/simulations'): os.mkdir('./data/simulations')
    for zip_file in os.listdir('data'):
        if zip_file.endswith('.zip'):
            print(" ----> " + zip_file)
            with zipfile.ZipFile("./data/" + zip_file, 'r') as zip_ref:
                zip_ref.extractall('./data/simulations')
    SIMULATIONS_PATH: str = "./data/simulations"

In [4]:
cmg   = np.load(SIMULATIONS_PATH + '/Channel_matrix_gain.npy')
alloc = np.load(SIMULATIONS_PATH + '/sisa-allocation.npy')

# get sample from all
n_sample = 12_000
cmg   = cmg[:n_sample]
alloc = alloc[:n_sample]

n_sample = cmg.shape[0]
K, N, _  = cmg.shape[1:]

shape    = lambda s: " x".join([f"{d:3d}" for d in s])
print(f"channel    matrix shape: {shape(cmg.shape)} \nallocation matrix shape: {shape(alloc.shape)}")

channel    matrix shape: 12000 x  4 x 20 x 20 
allocation matrix shape: 12000 x 20


## Sub-Band Assignment and Power Control for IoT Cellular Networks via Deep Learning

Two-stage deep learning optimization for sub-band assignment and transmit power control in uplink IoT networks using CNNs for improved sum rate and reduced computation time.

In [ ]:
# First Step: Subband allocation problem

class ConvBlock(nn.Module):
    def __init__(self, )